In [1]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize
from sklearn.model_selection import train_test_split
from gensim import corpora, models, similarities
from gensim.corpora import Dictionary
from math import sqrt

In [2]:
def make_category_1(file_path):


    f = open(file_path).read()
    data = json.loads(f)
    playlists = data["playlists"]
    df = pd.DataFrame(playlists)
    train, test = train_test_split(df, test_size=0.1)

    r_test_data = {'pid' : list(test['pid']), 'name' : list(test['name']), 'num_holdouts' : list(test['num_tracks']), 'tracks' : [{} for _ in range(test.shape[0])], 'num_samples' : [0 for _ in range(test.shape[0])], 'num_tracks' : test['num_tracks']}
    r_test = pd.DataFrame(r_test_data)
    
    train = train.T.to_dict().values()
    test = test.T.to_dict().values()
    r_test = r_test.T.to_dict().values()
    
    with open('./train.json', 'w+') as f:
        f.write(json.dumps(train))
    with open('./test_R.json', 'w+') as f:
        f.write(json.dumps(test))
    with open('./test.json', 'w+') as f:
        f.write(json.dumps(r_test))
        
        
make_category_1('mpd.slice.0-999.json')

<h1> train_set </h1>

In [3]:
f = open('./train.json').read()
data = json.loads(f)
train_df = pd.DataFrame(data)

<h1> test_set </h1>

In [4]:
f = open('./test_R.json').read()
data = json.loads(f)
test_df = pd.DataFrame(data)

<h1> cross validation test </h1>

In [13]:
train_playlist_name = list(set(train_df['name']))

train_playlist_name = [train_playlist_name[i:i + len(train_playlist_name)/10] 
                       for i in xrange(0, len(train_playlist_name), len(train_playlist_name)/10)]
topic_num = 3
max = 0
max_topic_num = 0
for idx in range(0, len(train_playlist_name)):
    #0~9 까지 validation
    validation_texts = [[word for word in name.lower().split()]for name in train_playlist_name[idx]]
    
    #train_set after slice
    train_texts = []
    for tidx in range(0,len(train_playlist_name)):
        if(tidx == idx):
            continue
        train_texts.extend(train_playlist_name[tidx])
    train_texts = [[word for word in name.lower().split()]
                  for name in train_texts]
    
    #LDA model create
    dictionary = corpora.Dictionary(train_texts)
    corpus = [dictionary.doc2bow(text) for text in train_texts]
    model = models.ldamodel.LdaModel(corpus = corpus, id2word = dictionary,
                                    num_topics = topic_num, random_state = 0,
                                    chunksize = 2000, passes = 10)
    
    #document 별 topic 다항분포 train
    train_dic = {}
    for idx in range(0, len(train_texts)):
        t = train_texts[idx]
        bow = dictionary.doc2bow(t)
        get_document_topics = tuple(model.get_document_topics(bow))
        name = ' '.join(t)
        train_dic[name] = dict(get_document_topics)
    train_dic = pd.DataFrame(train_dic)
    #print train_dic
    
    #document 별 topic 다항분포 validation
    validation_dic = {}
    for idx in range(0, len(validation_texts)):
        t = validation_texts[idx]
        bow = dictionary.doc2bow(t)
        get_document_topics = tuple(model.get_document_topics(bow))
        name = ' '.join(t)
        validation_dic[name] = dict(get_document_topics)
    validation_dic =pd.DataFrame(validation_dic).T
    #print validation_dic
    
    result = []
    for trainName in train_dic.columns:
        idx = 0
        #c = 0
        denominator = 0
        numeratorA = 0
        numeratorB = 0
        columns_ones = []
        for valName in validation_dic.index:
            for idx in range(0, topic_num):
                #c += pow(train_dic[trainName][idx] - validation_dic[idx][valName], 2)
                denominator += (train_dic[trainName][idx] * validation_dic[idx][valName])
                numeratorA += pow(train_dic[trainName][idx],2)
                numeratorB += pow(validation_dic[idx][valName],2)
            #c = 1/(1+sqrt(c))
            c = denominator / (sqrt(numeratorA) * sqrt(numeratorB))
            columns_ones.append(c)
        result.append(columns_ones)
    #train_playlist X validation_playlist_name 유클리디안 거리 행렬 생성    
    result = pd.DataFrame(result, columns = validation_dic.index, index = train_dic.columns)
    
    #validation_Set evaluation
    count = 0
    c = 0;
    for name in result.columns:
        #test_playlist_name 과 거리가 가까운 train_playlist_name 상위 10개 도출
        result_playlist_name = result[name].sort_values(ascending = False).head(10).index
        #print count
        count+=1
    
        #test_playlist 생성
        validation_playlist = []
        for idx in range(0, len(train_df['name'])):
            if(train_df['name'][idx].lower() == name):
                val_track_uri = pd.DataFrame(train_df['tracks'][idx])
                val_playlist = list(val_track_uri['track_uri'])
        #print len(test_playlist)        
        if(len(val_playlist) == 0):
            continue
            
        #비교할 train_playlist 생성
        train_playlist = []
        for train_name in result_playlist_name:
            for idx in range(0,len(train_df['name'])):
                if(train_df['name'][idx].lower() == train_name):
                    df = pd.DataFrame(train_df['tracks'][idx])
                    train_playlist += list(df['track_uri'])
                    if(len(train_playlist) >= 500):
                        continue
                    elif(len(train_playlist) + len(list(df['track_uri'])) >= 500):
                        temp = list(df['track_uri'])
                        train_playlist += temp[:-(len(train_playlist)-500)]
                    else:
                        train_playlist += list(df['track_uri'])
        #print len(train_playlist)      
        #r-precision 계산
        c += 1.0 *len(set(train_playlist).intersection(set(val_playlist)))/len(set(val_playlist)) *100
    
    #avarage result
    c = 1.0 * c/(count)
    print ("average : {}, topic_num : {}".format(c, topic_num))
    if max < c:
        max = c
        max_topic_num = topic_num
        maxModel = model

    topic_num += 2 #3,5,7,9,11,13,15,....
print ("max_average : {}, max_topic_num : {}".format(max, max_topic_num)) 

average : 5.95291695506, topic_num : 3
average : 6.11618295956, topic_num : 5
average : 6.11345528296, topic_num : 7
average : 6.75875662735, topic_num : 9
average : 5.95152494387, topic_num : 11
average : 7.86836192657, topic_num : 13
average : 6.79785158192, topic_num : 15
average : 10.205701679, topic_num : 17
average : 8.19271442626, topic_num : 19
average : 6.27356366582, topic_num : 21
average : 27.6587008294, topic_num : 23
max_average : 27.6587008294, max_topic_num : 23


<h1> test! </h1>

In [16]:
train_playlist_name = list(set(train_df['name']))
test_playlist_name = list(set(test_df['name']))

#train_playlist_name
#소문자로
train_texts = [[word for word in name.lower().split()]
         for name in train_playlist_name]

#test_playlist_name
#역시 소문자로
test_texts = [[word for word in name.lower().split()]
         for name in test_playlist_name]

topic_num = max_topic_num
 #LDA model create
#dictionary = corpora.Dictionary(train_texts)
#corpus = [dictionary.doc2bow(text) for text in train_texts]
#model = models.ldamodel.LdaModel(corpus = corpus, id2word = dictionary,
#                                   num_topics = topic_num, random_state = 0,
#                                    chunksize = 20000, passes = 10)
model = maxModel   
    #document 별 topic 다항분포 train
train_dic = {}
for idx in range(0, len(train_texts)):
    t = train_texts[idx]
    bow = dictionary.doc2bow(t)
    get_document_topics = tuple(model.get_document_topics(bow))
    name = ' '.join(t)
    train_dic[name] = dict(get_document_topics)
train_dic = pd.DataFrame(train_dic)
#print train_dic
    
    #document 별 topic 다항분포 validation
test_dic = {}
for idx in range(0, len(test_texts)):
    t = test_texts[idx]
    bow = dictionary.doc2bow(t)
    get_document_topics = tuple(model.get_document_topics(bow))
    name = ' '.join(t)
    test_dic[name] = dict(get_document_topics)
test_dic =pd.DataFrame(test_dic).T
#print validation_dic
    
result = []
for trainName in train_dic.columns:
    idx = 0
    c = 0
    columns_ones = []
    for testName in test_dic.index:
        for idx in range(0, topic_num):
            #c += pow(train_dic[trainName][idx] - test_dic[idx][testName], 2)
            denominator += (train_dic[trainName][idx] * test_dic[idx][testName])
            numeratorA += pow(train_dic[trainName][idx],2)
            numeratorB += pow(test_dic[idx][testName],2)
        #c = 1/(1+sqrt(c))
        c = 1.0 *denominator / (sqrt(numeratorA) * sqrt(numeratorB))
        columns_ones.append(c)
    result.append(columns_ones)
#train_playlist X validation_playlist_name 코사인 유사도 행렬 생성    
result = pd.DataFrame(result, columns = test_dic.index, index = train_dic.columns)
#print result   
    #validation_Set evaluation
count = 0
c = 0;
for name in result.columns:
    #test_playlist_name 과 거리가 가까운 train_playlist_name 상위 10개 도출
    result_playlist_name = result[name].sort_values(ascending = False).head(10).index
    print count
    
    #test_playlist 생성
    test_playlist = []
    for idx in range(0, len(test_df['name'])):
        if(test_df['name'][idx].lower() == name):
            test_track_uri = pd.DataFrame(test_df['tracks'][idx])
            test_playlist = list(test_track_uri['track_uri'])
    print "test_playlist length :{}".format(len(test_playlist))        
    if(len(test_playlist) == 0):
        continue
            
        #비교할 train_playlist 생성
    train_playlist = []
    for train_name in result_playlist_name:
        for idx in range(0,len(train_df['name'])):
            if(train_df['name'][idx].lower() == train_name):
                df = pd.DataFrame(train_df['tracks'][idx])
                temp =[]
                if(len(train_playlist) >= 500):
                    continue
                elif(len(train_playlist) + len(list(df['track_uri'])) >= 500):
                    temp = list(df['track_uri'])
                    train_playlist += temp[:-(len(train_playlist)-500)]
                else:
                    train_playlist += list(df['track_uri'])
    print "total_train_playlist : {}".format(len(train_playlist))      
    #r-precision 계산
    #if(len(set(train_playlist).intersection(set(test_playlist))) == 0):
    #    continue
    c += 1.0 *len(set(train_playlist).intersection(set(test_playlist)))/len(set(test_playlist)) *100
    count+=1
    print "r-precision summation: {}".format(c)
    print "count : {}".format(count)
    #avarage result
c = 1.0 * c/(count)
print "total average : {}".format(c)

0
test_playlist length :62
total_train_playlist : 500
r-precision summation: 4.91803278689
count : 1
1
test_playlist length :130
total_train_playlist : 500
r-precision summation: 4.91803278689
count : 2
2
test_playlist length :57
total_train_playlist : 500
r-precision summation: 8.42680471671
count : 3
3
test_playlist length :40
total_train_playlist : 500
r-precision summation: 8.42680471671
count : 4
4
test_playlist length :72
total_train_playlist : 500
r-precision summation: 18.1490269389
count : 5
5
test_playlist length :19
total_train_playlist : 500
r-precision summation: 23.4121848337
count : 6
6
test_playlist length :0
6
test_playlist length :29
total_train_playlist : 500
r-precision summation: 26.8604606957
count : 7
7
test_playlist length :0
7
test_playlist length :35
total_train_playlist : 500
r-precision summation: 32.57474641
count : 8
8
test_playlist length :53
total_train_playlist : 500
r-precision summation: 36.3483313157
count : 9
9
test_playlist length :111
total_train_

test_playlist length :205
total_train_playlist : 500
r-precision summation: 340.674195427
count : 80
80
test_playlist length :17
total_train_playlist : 500
r-precision summation: 346.556548368
count : 81
81
test_playlist length :16
total_train_playlist : 500
r-precision summation: 346.556548368
count : 82
82
test_playlist length :23
total_train_playlist : 500
r-precision summation: 346.556548368
count : 83
83
test_playlist length :26
total_train_playlist : 500
r-precision summation: 350.402702214
count : 84
84
test_playlist length :189
total_train_playlist : 500
r-precision summation: 352.006980289
count : 85
85
test_playlist length :150
total_train_playlist : 500
r-precision summation: 356.673646956
count : 86
86
test_playlist length :73
total_train_playlist : 500
r-precision summation: 356.673646956
count : 87
87
test_playlist length :29
total_train_playlist : 500
r-precision summation: 356.673646956
count : 88
total average : 4.0531096245
